In [75]:
import pandas as pd
import numpy as np
import torchviz
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import joblib
from sklearn.preprocessing import MinMaxScaler
import time
import warnings
warnings.simplefilter('ignore')
def get_col(df,name):
    return df.columns[df.columns.str.contains(name)].tolist()

# load data

In [76]:
tag = pd.read_csv('../data/tag_cleaned.csv',index_col=0).dropna(axis=0)
tag_map = dict(zip(tag.TAG,tag.chinese))
tag[tag.category=='可控制因子']

,item,TAG,chinese,character,unit,category,period,min,max,推測顯著性(依製程經驗)
0,1.0,MLPAP_TJC-0627.PV,OX預熱器出口溫度,溫度,℃,可控制因子,1分鐘2筆,145,160,***
1,2.0,MLPAP_TRC-0610.PV,空氣預熱器出口溫度,溫度,℃,可控制因子,1分鐘2筆,160,200,***
2,3.0,MLPAP_FR-0632.PV,汽機風量,流量,KNM3/HR,可控制因子,1分鐘2筆,64,85.4,****
3,4.0,MLPAP_FQ-0619.PV,OX入料量,流量,kg/HR,可控制因子,1分鐘2筆,2000,8000,****
4,5.0,MLPAP_TJC-0757A.PV,反應槽鹽浴溫度,溫度,℃,可控制因子,1分鐘2筆,345,365,****
5,6.0,MLPAP_PIC-0741.PV,汽鼓(V211)鍋爐水壓力,壓力,KG/CM2,可控制因子,1分鐘2筆,28,45,*
6,7.0,MLPAP_LIC-0738.PV,汽鼓(V211)鍋爐水液位,液位,%,可控制因子,1分鐘2筆,20,60,*
7,8.0,MLPAP_PIC-0761.PV,鹽浴冷卻器(E212)出口壓力,壓力,BAR,可控制因子,1分鐘2筆,40,55,***
9,10.0,MLPAP_LRC-0729.PV,鍋爐水液位控制,液位,%,可控制因子,1分鐘2筆,20,60,*
10,11.0,MLPAP_TJC-1721.PV,轉換式冷凝器冷油溫度,溫度,℃,可控制因子,1分鐘2筆,55,68,****


In [77]:
tag[tag.chinese.str.contains('環境溫度')]

,item,TAG,chinese,character,unit,category,period,min,max,推測顯著性(依製程經驗)
92,93.0,MLPAP_TJ3-1110.PV,環境溫度,溫度,℃,不可控制因子,1分鐘2筆,20,40,***


In [78]:
df = pd.read_csv('../data/train_cleaned.csv',index_col=0,parse_dates=True)
print(df.shape)
df.sample(5)

(228240, 96)


,MLPAP_TJ-0706B02,MLPAP_TJ-0706B03,MLPAP_TJ-0706B04,MLPAP_TJ-0706B05,MLPAP_TJ-0706B06,MLPAP_TJ-0706B07,MLPAP_TJ-0706B08,MLPAP_TJ-0706B09,MLPAP_TJ-0706B10,MLPAP_TJ-0706B11,...,MLPAP_FR-0791.PV,MLPAP_FI-V211.PV(轉換後),MLPAP_PI-0658.PV,MLPAP_TJC-0757.PV,MLPAP_V225-PHD,MLPAP_PI-0771.PV,MLPAP_TRC-0747.PV,MLPAP_TJ-0881.PV,MLPAP_TJ3-1110.PV,MLPAP_FQI-0997.SMH
2016-05-24 12:50:00,409.000000,427.000000,429.0,430.000000,418.0,412.0,400.000000,391.0,384.0,380.000000,...,26.708822,NaN,463.750412,0.600000,0.019,99.045490,179.950340,159.974663,27.759337,NaN
2016-10-01 07:10:00,381.000000,402.000000,409.0,415.000000,408.0,407.0,403.000000,397.0,386.0,378.000000,...,28.486210,NaN,410.000000,1.300000,0.036,110.598572,173.199997,138.899994,26.000000,NaN
2016-11-15 03:00:00,378.000000,396.000000,402.0,408.497086,404.0,404.0,405.000000,406.0,398.0,387.000000,...,31.692675,NaN,419.000000,2.149897,0.027,110.959949,172.200333,138.399994,24.200001,NaN
2019-05-21 16:00:00,380.322617,395.000000,398.0,403.000000,400.0,401.0,401.000000,399.0,398.0,395.000000,...,31.230598,6662.100830,NaN,1.032767,0.016,122.458179,169.100006,138.699997,26.100000,NaN
2019-03-26 06:10:00,382.000000,395.264557,397.0,400.000000,397.0,397.0,396.766891,394.0,392.0,389.119354,...,30.606353,6314.762451,456.000000,0.700000,0.006,111.975937,168.199997,137.973320,NaN,NaN


# add total_stream

In [79]:
stream_cols = tag[tag.chinese.str.contains('蒸氣')== True].TAG.tolist()
print(stream_cols)
df['stream'] = df[stream_cols[0]]*1000 + df[stream_cols[1]]*1000
df['stream'].describe()

['MLPAP_FR-0755.PV(轉換後)', 'MLPAP_FR-0791.PV', 'MLPAP_FI-V211.PV(轉換後)']


count    150576.000000
mean      43582.928582
std        5503.275059
min       14086.172342
25%       40355.246782
50%       45336.857319
75%       47993.137360
max       49753.607750
Name: stream, dtype: float64

# add new features

In [80]:
df['濃度'] = df['MLPAP_FQ-0619.PV']/df['MLPAP_FR-0632.PV']
df['濃度'].dropna().head()

2016-05-10 15:40:00    36.102136
2016-05-10 15:50:00    36.071262
2016-05-10 16:00:00    36.242898
2016-05-10 16:10:00    36.313928
2016-05-10 16:20:00    36.262361
Name: 濃度, dtype: float64

# define columns

In [81]:
action = tag[tag['category']=='可控制因子']['TAG'].values.tolist()
action = action + ['濃度']
action

['MLPAP_TJC-0627.PV',
 'MLPAP_TRC-0610.PV',
 'MLPAP_FR-0632.PV',
 'MLPAP_FQ-0619.PV',
 'MLPAP_TJC-0757A.PV',
 'MLPAP_PIC-0741.PV',
 'MLPAP_LIC-0738.PV',
 'MLPAP_PIC-0761.PV',
 'MLPAP_LRC-0729.PV',
 'MLPAP_TJC-1721.PV',
 'MLPAP_TRC-1734.PV',
 'MLPAP_TJC-0757.PV',
 'MLPAP_TRC-0747.PV',
 'MLPAP_TJ-0881.PV',
 'MLPAP_FQI-0997.SMH',
 'MLPAP_TJ-0757B.PV',
 '濃度']

In [82]:

# delete not need columns
action.remove('MLPAP_FQ-0619.PV')
action.remove('MLPAP_TJC-0757.PV')
action.remove('MLPAP_PIC-0741.PV')
action.remove('MLPAP_LIC-0738.PV')
action.remove('MLPAP_LRC-0729.PV')
action.insert(0,'MLPAP_FQ-0619.PV')
print('action:',len(action))

action: 13


In [83]:
action

['MLPAP_FQ-0619.PV',
 'MLPAP_TJC-0627.PV',
 'MLPAP_TRC-0610.PV',
 'MLPAP_FR-0632.PV',
 'MLPAP_TJC-0757A.PV',
 'MLPAP_PIC-0761.PV',
 'MLPAP_TJC-1721.PV',
 'MLPAP_TRC-1734.PV',
 'MLPAP_TRC-0747.PV',
 'MLPAP_TJ-0881.PV',
 'MLPAP_FQI-0997.SMH',
 'MLPAP_TJ-0757B.PV',
 '濃度']

In [84]:
action.remove('MLPAP_FQI-0997.SMH')

In [85]:
output = get_col(df,'0997')
output = [output[0]]
output

['MLPAP_FQI-0997.PV']

In [86]:
stream = ['stream']
print('stream:',len(stream))

stream: 1


In [87]:
state = get_col(df,'DATA OF USE') + ['MLPAP_TJ3-1110.PV']#環境溫度
print('state:',len(state))

state: 2


# select columns

In [88]:
df = df[state + action + output + stream]
print(df.shape)

(228240, 16)


# look miss value

In [89]:
#fig = plt.figure(figsize=(20,5))
#sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [90]:
df[action]['濃度'].describe()

count    128419.000000
mean         90.449640
std           6.346437
min          33.152719
25%          89.964452
50%          91.577896
75%          93.414036
max          96.211089
Name: 濃度, dtype: float64

# scale data

In [91]:
mm_state = MinMaxScaler()
mm_action = MinMaxScaler()
mm_output = MinMaxScaler()
mm_stream = MinMaxScaler()

df[state] = mm_state.fit_transform(df[state])
df[action] = mm_action.fit_transform(df[action])
df[output] = mm_output.fit_transform(df[output])
df[stream] = mm_stream.fit_transform(df[stream])

df.sample(5)

,DATA OF USE,MLPAP_TJ3-1110.PV,MLPAP_FQ-0619.PV,MLPAP_TJC-0627.PV,MLPAP_TRC-0610.PV,MLPAP_FR-0632.PV,MLPAP_TJC-0757A.PV,MLPAP_PIC-0761.PV,MLPAP_TJC-1721.PV,MLPAP_TRC-1734.PV,MLPAP_TRC-0747.PV,MLPAP_TJ-0881.PV,MLPAP_TJ-0757B.PV,濃度,MLPAP_FQI-0997.PV,stream
2020-04-11 23:10:00,0.904699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-24 09:40:00,0.951039,0.565883,0.729709,0.983347,0.703449,NaN,0.489362,0.943797,0.376366,0.336153,0.879921,0.229682,0.512821,NaN,0.777649,0.604541
2019-07-28 06:30:00,0.741459,0.481481,NaN,NaN,0.527522,NaN,NaN,NaN,0.351601,0.294069,0.413472,0.809187,NaN,NaN,NaN,NaN
2019-03-27 06:00:00,0.663831,0.068783,0.789087,0.732452,0.590353,0.55636,0.234042,0.955038,0.394317,0.302600,0.833735,0.208481,0.276924,0.867132,0.768573,0.625515
2019-08-03 19:40:00,0.745592,0.526176,NaN,NaN,0.506173,NaN,NaN,NaN,0.380810,0.865978,0.421225,0.837456,NaN,NaN,NaN,NaN


In [92]:
df[action]['濃度'].describe()

count    128419.000000
mean          0.908633
std           0.100644
min           0.000000
25%           0.900939
50%           0.926525
75%           0.955643
max           1.000000
Name: 濃度, dtype: float64

In [93]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
# Missing values statistics
missing_values = missing_values_table(df)
missing_values.head(20)

Your selected dataframe has 16 columns.
There are 16 columns that have missing values.


,Missing Values,% of Total Values
MLPAP_TJC-0627.PV,103249,45.2
濃度,99821,43.7
MLPAP_FR-0632.PV,99787,43.7
MLPAP_FQI-0997.PV,77845,34.1
stream,77664,34.0
MLPAP_TJ-0757B.PV,70136,30.7
MLPAP_TJC-0757A.PV,70059,30.7
MLPAP_FQ-0619.PV,69183,30.3
MLPAP_TJ3-1110.PV,66331,29.1
MLPAP_PIC-0761.PV,59989,26.3


# build dataset for modeling

In [94]:
def build_dataset(df,n):
    
    data = {'state':[],
            'action':[],
            'output':[],
            'stream':[],
            'idx':[],
           }
    
    for i in tqdm(range(0,len(df)-n)):
        curr = df.iloc[i:i+n] #一小段資料
        if np.isnan(curr.values).sum() == 0: #確保這段資料無缺失值
            
            data['state'].append(curr[state].values.mean(axis=0)) #這段時間的 "狀態" 之均值
            data['action'].append(curr[action].values)# 這段時間 "所有" 操作
            data['output'].append(curr[output].values[-1])#這段時間 最後一個時間點 的出料(delay issue)
            data['stream'].append(curr[stream].values[-1])#這段時間 最後一個時間點 的蒸氣(delay issue)
            data['idx'].append(curr.index[-1]) #這段時間最後 出料對應的時間點
    
    # reshape
    data['state'] = np.array(data['state'])
    data['action'] = np.array(data['action'])
    data['output'] = np.array(data['output']).reshape(-1,1)
    data['stream'] = np.array(data['stream']).reshape(-1,1)
    
    # store mm_scaler
    data['mm_state'] = mm_state
    data['mm_action'] = mm_action
    data['mm_output'] = mm_output
    data['mm_stream'] = mm_stream
    
    # store col_name
    data['state_col'] = state
    data['action_col'] = action
    data['output_col'] = output
    data['stream_col'] = stream
    
    return data

In [95]:
data = build_dataset(df,36)

In [96]:
print(len(data['idx']))

59213


# save

In [97]:
joblib.dump(data,'../data/dataset.pkl')

['../data/dataset.pkl']